# Redes Neurais nos Exames de Bruxo de Hogwarts

### Descriçao

---
1. O objetivo deste notebook é aprender sobre Redes Neurais Artificias
2. Vamos construir um conjunto de dados baseado no universo do Harry Potter
3. O problema consiste em prever qual a nota do aluno
---

### Dicionário


Fields	                                                  | Type  	  |    Description                              |
----------------------------------------------------------|:---------:|:-------------------------------------------:|
student 	  										  	  |string     | name of participants	                    |
age														  |integer    | age of participants                         |
houses		     										  |string     | house of participants		                |
spell												  |float      | subject									    |
hours_of_studie											  |integer    | time spend in studie	     			    |
hours_of_practice										 |integer    | time spend in practice	     			    |
alchemy            										  |float     | subject                                     |


----
### Objetivos

1. O problema consiste em criar uma rede neural capaz de prever a nota que o aluno vai tirar em determinada materia.
2. Vamos construir um conjunto de dados historicos desses alunos e dividi-los em teste e treino.
3. Variaveis independentes (Horas de Estudo e Horas de Pratica)
4. Variavel alvo (Qualquer materia)
---


### Baixando os pacotes

In [ ]:
!pip install pandas numpy scikit-learn keras Faker


### Documentação

1. ** Pandas ** -> [Link](https://pandas.pydata.org/docs/)
2. ** Numpy ** -> [Link](https://numpy.org/doc/)
3. ** Faker ** -> [Link](https://faker.readthedocs.io/en/master/)
4. ** Scikit Learn ** -> [Link](https://scikit-learn.org/stable/)
5. ** Keras ** -> [Link](https://keras.io/api/)
6. ** Tensor Flow ** -> [Tensor Flow](https://www.tensorflow.org/api_docs/python/tf/keras)


In [ ]:
import pandas as pd # para manipulacao de dados
import numpy as np # para manipulação de matrizes
import random #para gerar numeros aleatorios
from faker import Faker # para gerar nomes falsos
from sklearn.model_selection import train_test_split #para dividir os dados em teste e treino
from sklearn.preprocessing import StandardScaler # etapa de pre processamento de dados para deixar dados na mesma escala
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score #metricas de avaliacao
from keras.models import Sequential # para redes neurais
from keras.layers import Dense, Dropout # para etapas de ativação de camadas neurais
from tensorflow.keras.utils import plot_model # para visualizar as camadas neurais

# Construindo o Dataset Faker

In [ ]:

def generate(num):
    fake = Faker() #criando o metodo de nomes falsos
    data = []
    subjects = ['spell', 'alchemy'] #lista de materias

    for _ in range(num):
        student = fake.name()
        age = random.randint(11, 39)
        house = random.choice(['Gryffindor', 'Slytherin', 'Ravenclaw', 'Hufflepuff'])
        hours_of_studie = random.randint(1,24)
        hours_of_practice = random.randint(1, 24)
        row = {
            'student': student,
            'age': age,
            'house': house,
            'hours_of_studie': hours_of_studie,
            'hours_of_practice': hours_of_practice,
        }
        for subject in subjects:
            row[subject] = round(random.uniform(0,10),2)
        data.append(row)

    df = pd.DataFrame(data)
    return df

df = generate(1000)
df


### Criando nosso modelo de rede neural

In [ ]:
df['house'].value_counts()

In [ ]:
#para as casas do harry potter vamos atribuir valores numericos, o computador nao processa texto
# poderiamos tambem usar o LabelEnconder, da biblioteca do Scikit Learn

house_map = {'Gryffindor': 0, 'Slytherin': 1, 'Ravenclaw': 2, 'Hufflepuff': 3}
df['house'] = df['house'].map(house_map)

In [ ]:
df['house'].value_counts()

In [ ]:
# Definir as variáveis independentes e a variável alvo
X = df[['hours_of_studie', 'hours_of_practice', 'age', 'house']] # o X é a variavel indepedente
y = df['spell']  # o 'y' é a variável alvo

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.head(4)

In [ ]:
y_train.head(4)

In [ ]:
# Normalizar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
X_train

In [ ]:
# Construir o modelo da rede neural
modelo = Sequential()
modelo.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
modelo.add(Dropout(0.4))
modelo.add(Dense(units=32, activation='relu'))
modelo.add(Dropout(0.4))
modelo.add(Dense(units=64, activation='relu'))
modelo.add(Dropout(0.4))
modelo.add(Dense(units=1, activation='sigmoid'))


In [ ]:
# Compilar o modelo
modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:

# Treinar o modelo
modelo.fit(X_train, y_train, epochs=50, batch_size=32)

In [ ]:
# Avaliar o modelo
loss = modelo.evaluate(X_test, y_test)
print(f'Loss: {loss}')

In [ ]:
# Fazer previsões
predictions = modelo.predict(X_test)

### Visualizando nossa rede neural

In [ ]:
plot_model(modelo, to_file='modelo.png', rankdir="LR", show_shapes=True, show_layer_names=True)

### Testando nosso modelo

In [ ]:
df.head(4)

# Confirmando nossa classificação

In [ ]:
# Remover as colunas 'age' e 'house' do DataFrame de teste
test_data = df[['hours_of_studie', 'hours_of_practice', 'age', 'house']]

# Normalizar os dados de teste
X_test = scaler.transform(test_data)


# Comparar previsões com as notas reais
for i, prediction in enumerate(predictions):
    print(f'Previsão: {prediction[0]}, Nota Real: {df.iloc[i]["spell"]}')


# Confirmando com novos valores

In [ ]:
new_df = generate(4)
house_map = {'Gryffindor': 0, 'Slytherin': 1, 'Ravenclaw': 2, 'Hufflepuff': 3}
new_df['house'] = new_df['house'].map(house_map)
new_df

In [ ]:

X_test = scaler.transform(new_df[['hours_of_studie', 'hours_of_practice', 'age', 'house']]) # Normalizar os novos dados


# Gerar previsões
predictions = modelo.predict(X_test)

# Exibir as previsões
for i, prediction in enumerate(predictions):
    print(f'Previsão: {prediction[0]}, Nota Real: {new_df.iloc[i]["spell"]}')

### Avaliando o Modelo

In [ ]:
X_all = scaler.transform(df[['hours_of_studie', 'hours_of_practice']])
predictions_all = modelo.predict(X_all)

y_true = df['spell']  # ou 'alchemy' como variável alvo
y_pred = predictions_all.flatten()

# Calcular o erro médio absoluto (MAE)
mae = mean_absolute_error(y_true, y_pred)

# Calcular o erro quadrático médio (MSE)
mse = mean_squared_error(y_true, y_pred)

# Calcular o coeficiente de determinação (R²)
r2 = r2_score(y_true, y_pred)

print(f'Erro Médio Absoluto (MAE): {mae}') # O MAE é a média das diferenças absolutas entre as previsões do modelo e os valores reais. Em outras palavras, ele mede o quão perto as previsões estão dos valores reais, sem considerar a direção. Um MAE mais baixo indica que o modelo está fazendo previsões mais precisas.
print(f'Erro Quadrático Médio (MSE): {mse}') # O MSE é a média dos quadrados das diferenças entre as previsões do modelo e os valores reais. Ele dá mais peso a grandes erros, o que pode ser útil em alguns casos.
print(f'Coeficiente de Determinação (R²): {r2}') # O R² é uma medida estatística que indica a proporção da variância nos valores da variável dependente que é previsível a partir dos valores da variável independente. Em outras palavras, ele indica o quão bem os novos dados devem se encaixar no modelo.


### Inferencia

um MAE de aproximadamente 4.13 significa que, em média, as previsões do seu modelo estão a cerca de 4.13 unidades de distância dos valores reais. Um MSE de aproximadamente 24.51 indica que, em média, o quadrado da diferença entre as previsões e os valores reais é de aproximadamente 24.51. O coeficiente de determinação negativo indica que o modelo não se ajustou bem aos dados, sugerindo que há outros fatores não considerados pelo modelo que afetam as notas dos alunos.